In [32]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as td
from tensorflow.keras import layers,Model

In [12]:
df=pd.read_csv("/home/saurav-neupane/full_codes/dynamic-ride-pricing/data/processed/train_features.csv")

In [13]:
df.head()

,fare_amount,passenger_count,hour,day_of_week,is_weekend,is_peak_hour,distance_km
0,4.5,1,17,0,0,1,1.030764
1,16.9,1,16,1,0,1,8.450134
2,5.7,2,0,3,0,0,1.389525
3,7.7,1,4,5,1,0,2.799270
4,5.3,1,7,1,0,1,1.999157


In [14]:
X=df.drop(columns=["fare_amount"])
y=df["fare_amount"]

In [15]:
X_train, X_test, y_train, y_test=train_test_split(X,y, random_state=42, test_size=0.2)

In [16]:
wide_features=["is_peak_hour","is_weekend","passenger_count"]
deep_features=["distance_km","hour","day_of_week"]

X_train_wide=X_train[wide_features]
X_train_deep=X_train[deep_features]
X_test_wide=X_test[wide_features]
X_test_deep=X_test[deep_features]

In [20]:
wide_input=layers.Input(shape=(X_train_wide.shape[1],),name="wide_input")
wide_output=layers.Dense(1)(wide_input)

2026-01-03 21:29:14.785478: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [23]:
deep_input=layers.Input(shape=(X_train_deep.shape[1],), name="deep_input")
x1=layers.Dense(64, activation="relu")(deep_input)
x2=layers.Dense(32, activation="relu")(x1)
deep_output=layers.Dense(1)(x2)

In [26]:
combined=layers.Add()([wide_output, deep_output])
model=Model(
    inputs=[wide_input, deep_input],
    outputs=combined
)

In [33]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

In [34]:
history = model.fit(
    [X_train_wide, X_train_deep],
    y_train,
    validation_split=0.1,
    epochs=10,
    batch_size=1024,
    verbose=1
)

Epoch 1/10
37763/37763 ━━━━━━━━━━━━━━━━━━━━ 35s 864us/step - loss: 494.5452 - mae: 2.2541 - val_loss: 20.0072 - val_mae: 2.2335
Epoch 2/10
37763/37763 ━━━━━━━━━━━━━━━━━━━━ 34s 861us/step - loss: 494.5178 - mae: 2.2526 - val_loss: 19.8991 - val_mae: 2.2407
Epoch 3/10
37763/37763 ━━━━━━━━━━━━━━━━━━━━ 34s 867us/step - loss: 494.5142 - mae: 2.2528 - val_loss: 19.9426 - val_mae: 2.2587
Epoch 4/10
37763/37763 ━━━━━━━━━━━━━━━━━━━━ 34s 873us/step - loss: 494.4982 - mae: 2.2509 - val_loss: 19.8994 - val_mae: 2.2367
Epoch 5/10
37763/37763 ━━━━━━━━━━━━━━━━━━━━ 34s 869us/step - loss: 494.4809 - mae: 2.2488 - val_loss: 19.9105 - val_mae: 2.2400
Epoch 6/10
37763/37763 ━━━━━━━━━━━━━━━━━━━━ 34s 876us/step - loss: 494.4736 - mae: 2.2484 - val_loss: 19.9178 - val_mae: 2.2331
Epoch 7/10
37763/37763 ━━━━━━━━━━━━━━━━━━━━ 35s 882us/step - loss: 494.4707 - mae: 2.2490 - val_loss: 19.9174 - val_mae: 2.2591
Epoch 8/10
37763/37763 ━━━━━━━━━━━━━━━━━━━━ 34s 870us/step - loss: 494.4850 - mae: 2.2501 - val_loss: 19

In [35]:
y_pred = model.predict([X_test_wide, X_test_deep]).ravel()

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))


335668/335668 ━━━━━━━━━━━━━━━━━━━━ 138s 410us/step


In [36]:
mae

2.3063306121638205

In [37]:
rmse

np.float64(5.248956144079253)

In [39]:
model.save("/home/saurav-neupane/full_codes/dynamic-ride-pricing/models/wide_and_deep_v1.keras")
